<a href="https://colab.research.google.com/github/shahd1995913/Machine-Learning/blob/main/text_classification_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
import nltk
import sklearn
import pandas
import numpy

print('Python: {}'.format(sys.version))
print('NLTK: {}'.format(nltk.__version__))
print('Scikit-learn: {}'.format(sklearn.__version__))
print('Pandas: {}'.format(pandas.__version__))
print('Numpy: {}'.format(numpy.__version__))

Python: 3.6.9 (default, Oct  8 2020, 12:12:24) 
[GCC 8.4.0]
NLTK: 3.2.5
Scikit-learn: 0.22.2.post1
Pandas: 1.1.4
Numpy: 1.18.5


In [ ]:
from google.colab import files
uploaded =files.upload()
for fn in uploaded.keys():
  print('User uploaeded file "{name}" with length {length} bytes'.format(
      name=fn,length=len(uploaded[fn])))

Saving A1.xlsx to A1.xlsx
User uploaeded file "A1.xlsx" with length 2394045 bytes


In [ ]:
import numpy as np
import pandas as pd
df = pd.read_excel('A1.xlsx', index_col=0)
print(df)

                     Review_Date Author_Name  ... result     Label
0     on 05/02/18 12:10 PM (PDT)        Shaf  ...      1   Postive
1     on 04/10/18 19:01 PM (PDT)    RogerCVC  ...     -1  Negative
2     on 10/24/17 18:33 PM (PDT)        Mike  ...      1   Postive
3     on 08/08/17 19:30 PM (PDT)     Jacob H  ...     -1  Negative
4     on 03/17/17 15:05 PM (PDT)       d. b.  ...      1   Postive
...                          ...         ...  ...    ...       ...
8053  on 10/18/05 22:23 PM (PDT)        Nick  ...      1   Postive
8054  on 03/24/11 11:52 AM (PDT)      m3isme  ...      1   Postive
8055  on 09/15/10 00:00 AM (PDT)       JORGE  ...      1   Postive
8056  on 05/27/10 14:20 PM (PDT)      Walter  ...      1   Postive
8057  on 11/28/16 21:17 PM (PST)    M5 owner  ...      1   Postive

[7414 rows x 8 columns]


In [ ]:
import nltk
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import *

In [ ]:
# print useful information about the dataset
print(df.info())
print(df.head())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7414 entries, 0 to 8057
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Review_Date    7414 non-null   object 
 1   Author_Name    7412 non-null   object 
 2   Vehicle_Title  7414 non-null   object 
 3   Review_Title   7413 non-null   object 
 4   Review         7365 non-null   object 
 5   Rating         7412 non-null   float64
 6   result         7414 non-null   int64  
 7   Label          7414 non-null   object 
dtypes: float64(1), int64(1), object(6)
memory usage: 521.3+ KB
None
                  Review_Date Author_Name  ... result     Label
0  on 05/02/18 12:10 PM (PDT)        Shaf  ...      1   Postive
1  on 04/10/18 19:01 PM (PDT)    RogerCVC  ...     -1  Negative
2  on 10/24/17 18:33 PM (PDT)        Mike  ...      1   Postive
3  on 08/08/17 19:30 PM (PDT)     Jacob H  ...     -1  Negative
4  on 03/17/17 15:05 PM (PDT)       d. b.  ...      1   Postive



In [ ]:
# check class distribution
classes = df['Label']
print(classes.value_counts())

Postive     6751
Negative     663
Name: Label, dtype: int64


In [ ]:
from sklearn.preprocessing import LabelEncoder

# convert class labels to binary values, 0 = ham and 1 = spam
encoder = LabelEncoder()
Y = encoder.fit_transform(classes)

print(Y[:10])

[1 0 1 0 1 1 1 1 1 0]


In [ ]:
# store the SMS message data
text_messages = df['Review']
print(text_messages[:10])


0                     Beautiful car.  Love all aspects
1     We picked this car as a CPO from a BMW dealer...
2    Get ride of the run flats.   If you like to dr...
3     1.\tDealership was a joke to work with, lied ...
4                        Handy as a pocket on a shirt.
5     Drives like a nice little SUV sports car. I w...
6    35 mpg on a 3000 mile trip 24+intown in winter...
7     Purchased 2014 X1 fully loaded with M-Sport p...
8    The great handling and braking of a 3 series w...
9     My first BMW and I'm now caught in the Takata...
Name: Review, dtype: object


In [ ]:
# use regular expressions to replace email addresses, URLs, phone numbers, other numbers

# Replace email addresses with 'email'
processed = text_messages.str.replace(r'^.+@[^\.].*\.[a-z]{2,}$',
                                 'emailaddress')

# Replace URLs with 'webaddress'
processed = processed.str.replace(r'^http\://[a-zA-Z0-9\-\.]+\.[a-zA-Z]{2,3}(/\S*)?$',
                                  'webaddress')

# Replace money symbols with 'moneysymb' (£ can by typed with ALT key + 156)
processed = processed.str.replace(r'£|\$', 'moneysymb')
    
# Replace 10 digit phone numbers (formats include paranthesis, spaces, no spaces, dashes) with 'phonenumber'
processed = processed.str.replace(r'^\(?[\d]{3}\)?[\s-]?[\d]{3}[\s-]?[\d]{4}$',
                                  'phonenumbr')
    
# Replace numbers with 'numbr'
processed = processed.str.replace(r'\d+(\.\d+)?', 'numbr')

In [ ]:
# Remove punctuation
processed = processed.str.replace(r'[^\w\d\s]', ' ')

# Replace whitespace between terms with a single space
processed = processed.str.replace(r'\s+', ' ')

# Remove leading and trailing whitespace
processed = processed.str.replace(r'^\s+|\s+?$', '')

In [ ]:
# change words to lower case - Hello, HELLO, hello are all the same word
processed = processed.str.lower()
print(processed)

0                          beautiful car love all aspects
1       we picked this car as a cpo from a bmw dealer ...
2       get ride of the run flats if you like to drink...
3       numbr dealership was a joke to work with lied ...
4                            handy as a pocket on a shirt
                              ...                        
8053    this car is a very well rounded car offers all...
8054    have used my friends for a month we traded i h...
8055    i do have the pleasure to tell you guys that i...
8056    this is my fifth bmw and third m car first mnu...
8057    so far so good had the car for about a week fa...
Name: Review, Length: 7414, dtype: object


In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

# remove stop words from text messages

stop_words = set(stopwords.words('english'))

processed = processed.apply(lambda x: " ".join(x.lower() for x in str(x).split() \
                                    if x not in stop_words))
print(processed)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
0                              beautiful car love aspects
1       picked car cpo bmw dealer problems date relati...
2                get ride run flats like drink coffee car
3       numbr dealership joke work lied two separate o...
4                                      handy pocket shirt
                              ...                        
8053    car well rounded car offers sorts modes option...
8054    used friends month traded mnumbr great car man...
8055    pleasure tell guys get one never ever day with...
8056    fifth bmw third car first mnumbr ive numbr ser...
8057    far good car week family numbr fits comfortabl...
Name: Review, Length: 7414, dtype: object


In [ ]:
# Remove word stems using a Porter stemmer
ps = nltk.PorterStemmer()

processed = processed.apply(lambda x: ' '.join(
    ps.stem(term) for term in x.split()))
print(processed)

0                                  beauti car love aspect
1       pick car cpo bmw dealer problem date rel decen...
2                   get ride run flat like drink coff car
3       numbr dealership joke work lie two separ occa ...
4                                      handi pocket shirt
                              ...                        
8053    car well round car offer sort mode option fit ...
8054    use friend month trade mnumbr great car mani c...
8055    pleasur tell guy get one never ever day withou...
8056    fifth bmw third car first mnumbr ive numbr ser...
8057    far good car week famili numbr fit comfort ent...
Name: Review, Length: 7414, dtype: object


In [ ]:
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
# create bag-of-words
all_words = []

for message in processed:
    words = word_tokenize(message)
    for w in words:
        all_words.append(w)
        
all_words = nltk.FreqDist(all_words)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# print the total number of words and the 15 most common words
print('Number of words: {}'.format(len(all_words)))
print('Most common words: {}'.format(all_words.most_common(15)))

Number of words: 10489
Most common words: [('numbr', 18265), ('car', 14241), ('drive', 6461), ('bmw', 5484), ('great', 3097), ('get', 3025), ('like', 2963), ('mile', 2522), ('one', 2510), ('handl', 2311), ('love', 2217), ('year', 2198), ('look', 2088), ('sport', 1898), ('new', 1802)]


In [ ]:
# use the 1500 most common words as features
word_features = list(all_words.keys())[:1500]

In [ ]:
# The find_features function will determine which of the 1500 word features are contained in the review
def find_features(message):
    words = word_tokenize(message)
    features = {}
    for word in word_features:
        features[word] = (word in words)

    return features

# Lets see an example!
features = find_features(processed[0])
for key, value in features.items():
    if value == True:
        print(key)

beauti
car
love
aspect


In [ ]:
# Now lets do it for all the messages
messages = list(zip(processed, Y))

# define a seed for reproducibility
seed = 1
np.random.seed = seed
np.random.shuffle(messages)

# call find_features function for each SMS message
featuresets = [(find_features(text), label) for (text, label) in messages]

In [ ]:
# we can split the featuresets into training and testing datasets using sklearn
from sklearn import model_selection

# split the data into training and testing datasets
training, testing = model_selection.train_test_split(featuresets, test_size = 0.25, random_state=seed)

In [ ]:
print(len(training))
print(len(testing))

5560
1854


In [ ]:
# We can use sklearn algorithms in NLTK
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.svm import SVC

model = SklearnClassifier(SVC(kernel = 'linear'))

# train the model on the training data
model.train(training)

# and test on the testing dataset!
accuracy = nltk.classify.accuracy(model, testing)*100
print("SVC Accuracy: {}".format(accuracy))

SVC Accuracy: 89.5900755124056


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

# Define models to train
names = ["K Nearest Neighbors", "Decision Tree", "Random Forest", "Logistic Regression", "SGD Classifier",
         "Naive Bayes", "SVM Linear"]

classifiers = [
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    LogisticRegression(),
    SGDClassifier(max_iter = 100),
    MultinomialNB(),
    SVC(kernel = 'linear')
]

models = zip(names, classifiers)

for name, model in models:
    nltk_model = SklearnClassifier(model)
    nltk_model.train(training)
    accuracy = nltk.classify.accuracy(nltk_model, testing)*100
    print("{} Accuracy: {}".format(name, accuracy))

K Nearest Neighbors Accuracy: 84.2502696871629
Decision Tree Accuracy: 86.46170442286947
Random Forest Accuracy: 91.8015102481122
Logistic Regression Accuracy: 91.47788565264293
SGD Classifier Accuracy: 90.1294498381877
Naive Bayes Accuracy: 88.45738942826321
SVM Linear Accuracy: 89.5900755124056


In [ ]:
# Ensemble methods - Voting classifier
from sklearn.ensemble import VotingClassifier

names = ["K Nearest Neighbors", "Decision Tree", "Random Forest", "Logistic Regression", "SGD Classifier",
         "Naive Bayes", "SVM Linear"]

classifiers = [
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    LogisticRegression(),
    SGDClassifier(max_iter = 100),
    MultinomialNB(),
    SVC(kernel = 'linear')
]

models = list(zip(names, classifiers))

nltk_ensemble = SklearnClassifier(VotingClassifier(estimators = models, voting = 'hard', n_jobs = -1))
nltk_ensemble.train(training)
accuracy = nltk.classify.accuracy(nltk_model, testing)*100
print("Voting Classifier: Accuracy: {}".format(accuracy))

Voting Classifier: Accuracy: 89.5900755124056


In [ ]:
# make class label prediction for testing set
txt_features, labels = zip(*testing)

prediction = nltk_ensemble.classify_many(txt_features)

In [ ]:
# print a confusion matrix and a classification report
print(classification_report(labels, prediction))

pd.DataFrame(
    confusion_matrix(labels, prediction),
    index = [['actual', 'actual'], ['Postive', 'Negative']],
    columns = [['predicted', 'predicted'], ['Postive', 'Negative']])

              precision    recall  f1-score   support

           0       0.52      0.34      0.41       155
           1       0.94      0.97      0.96      1699

    accuracy                           0.92      1854
   macro avg       0.73      0.66      0.68      1854
weighted avg       0.91      0.92      0.91      1854



predicted         
                  Postive Negative
actual Postive         53      102
       Negative        49     1650